***
# Making a new dataset from the Kaggle dataset (only keeping ['sadness', 'anger', 'love', 'surprise', 'neutral'])
Will use as another "test" dataset on the existing models
***

***
## 1 Load the dataset

In [1]:
import pandas as pd
import numpy as np
import random

from google.colab import drive 
drive.mount('/content/gdrive')

project_folder = '/content/gdrive/MyDrive/266_Final_Project_Folder'

# read from the google drive
source = project_folder + '/datasets/tweet_emotions.csv'

# Load the csv
df = pd.read_csv(source)

print(df.head())
print(len(df))

Mounted at /content/gdrive
     tweet_id   sentiment                                            content
0  1956967341       empty  @tiffanylue i know  i was listenin to bad habi...
1  1956967666     sadness  Layin n bed with a headache  ughhhh...waitin o...
2  1956967696     sadness                Funeral ceremony...gloomy friday...
3  1956967789  enthusiasm               wants to hang out with friends SOON!
4  1956968416     neutral  @dannycastillo We want to trade with someone w...
40000


In [2]:
# get duplicate records based on the 'text' column
duplicate_df = df[df.duplicated(subset='content', keep=False)].sort_values('content')

duplicate_df

tweet_id sentiment                                            content
31332  1751919171   neutral  #frenchieb-day #frenchieb-day #frenchieb-day #...
31363  1751920439   neutral  #frenchieb-day #frenchieb-day #frenchieb-day #...
30929  1751783552   neutral  #frenchieb-day #frenchieb-day #frenchieb-day #...
15028  1964712204   neutral                                                  0
29869  1751394197   neutral                                                  0
...           ...       ...                                                ...
14067  1964326790  surprise  we're getting another puppy  ... not cool... r...
6954   1961495657   neutral                                 why am i so tired?
51     1956979900     worry                                 why am i so tired?
8241   1962103517    relief                                       work all day
6023   1961126132   neutral                                       work all day

[262 rows x 3 columns]

In [3]:
df.shape

(40000, 3)

In [4]:
duplicate_df.shape

(262, 3)

In [5]:
df.shape[0] - duplicate_df.shape[0]

39738

In [6]:
# create boolean mask for rows in df where the value 
# in the 'text' column is also in duplicate_df
mask = df['content'].isin(duplicate_df['content'])

# exclude rows in df where the 'text' value is also in duplicate_df
df = df[~mask]

df.shape

(39738, 3)

In [7]:
df = df.rename(columns={'content': 'text', 'sentiment': 'emotions'})
df = df.drop('tweet_id', axis=1)
df

emotions                                               text
0           empty  @tiffanylue i know  i was listenin to bad habi...
1         sadness  Layin n bed with a headache  ughhhh...waitin o...
2         sadness                Funeral ceremony...gloomy friday...
3      enthusiasm               wants to hang out with friends SOON!
4         neutral  @dannycastillo We want to trade with someone w...
...           ...                                                ...
39995     neutral                                   @JohnLloydTaylor
39996        love                     Happy Mothers Day  All my love
39997        love  Happy Mother's Day to all the mommies out ther...
39998   happiness  @niariley WASSUP BEAUTIFUL!!! FOLLOW ME!!  PEE...
39999        love  @mopedronin bullet train from tokyo    the gf ...

[39738 rows x 2 columns]

***
## 2 Basic EDA

### Datatypes for each of the columns in our dataframe

In [8]:
print(df.dtypes)

emotions    object
text        object
dtype: object


### Unique types and count of sentiments in the dataset

In [9]:
# returns a dataframe with emotions as rows and count of those emotions and percentage of total in that df
def get_proportions(data):
  # get the value counts of each unique value
  counts = data.value_counts()

  # convert the counts to percentages
  percentages = counts / counts.sum() * 100

  return pd.DataFrame({'count': counts, 'percentage': percentages})  

In [10]:
# original (with dupes)
cp_df = get_proportions(df['emotions'])
print(cp_df)

            count  percentage
neutral      8580   21.591424
worry        8421   21.191303
happiness    5169   13.007700
sadness      5142   12.939755
love         3770    9.487141
surprise     2177    5.478383
fun          1774    4.464241
relief       1516    3.814988
hate         1322    3.326790
empty         821    2.066033
enthusiasm    757    1.904978
boredom       179    0.450450
anger         110    0.276813


In [11]:
desired_emotions = ['sadness', 'anger', 'love', 'surprise', 'neutral']
new_df = df[df['emotions'].isin(desired_emotions)]
new_df

emotions                                               text
1      sadness  Layin n bed with a headache  ughhhh...waitin o...
2      sadness                Funeral ceremony...gloomy friday...
4      neutral  @dannycastillo We want to trade with someone w...
6      sadness  I should be sleep, but im not! thinking about ...
8      sadness            @charviray Charlene my love. I miss you
...        ...                                                ...
39993  neutral  @sendsome2me haha, yeah. Twitter has many uses...
39995  neutral                                   @JohnLloydTaylor
39996     love                     Happy Mothers Day  All my love
39997     love  Happy Mother's Day to all the mommies out ther...
39999     love  @mopedronin bullet train from tokyo    the gf ...

[19779 rows x 2 columns]

In [12]:
proportions_df = get_proportions(new_df['emotions'])
print(proportions_df)

          count  percentage
neutral    8580   43.379342
sadness    5142   25.997270
love       3770   19.060620
surprise   2177   11.006623
anger       110    0.556145


### Make csv of kaggle dataset

In [13]:
new_df.to_csv(project_folder + '/datasets/kaggle_data_no_dupe.csv')

# Combine emotions:
"hate" >> "anger"<br>
"worry" >> "fear"<br>
"happiness" >> "joy"<br>
"fun" >> "joy"

In [14]:
df_squished_emotion = df.copy(deep=True)
df_squished_emotion.shape

(39738, 2)

In [15]:
# replace 'hate' with 'anger' in 'emotions' column
df_squished_emotion['emotions'] = df_squished_emotion['emotions'].replace('hate', 'anger')

# replace 'worry' with 'fear' in 'emotions' column
df_squished_emotion['emotions'] = df_squished_emotion['emotions'].replace('worry', 'fear')

# replace 'happiness' with 'joy' in 'emotions' column
df_squished_emotion['emotions'] = df_squished_emotion['emotions'].replace('happiness', 'joy')

# replace 'fun' with 'joy' in 'emotions' column
df_squished_emotion['emotions'] = df_squished_emotion['emotions'].replace('fun', 'joy')

In [16]:
df_squished_emotion.shape

(39738, 2)

In [17]:
# original (with dupes)
squished_proportions = get_proportions(df_squished_emotion['emotions'])
print(squished_proportions)

            count  percentage
neutral      8580   21.591424
fear         8421   21.191303
joy          6943   17.471941
sadness      5142   12.939755
love         3770    9.487141
surprise     2177    5.478383
relief       1516    3.814988
anger        1432    3.603604
empty         821    2.066033
enthusiasm    757    1.904978
boredom       179    0.450450


In [18]:
# class_to_index_dict = {'sadness': 0, 'anger': 1, 'fear': 2, 'surprise': 3, 'joy': 4, 'love': 5, 'neutral': 6}
desired_emotions = ['sadness', 'anger', 'fear', 'surprise', 'joy', 'love', 'neutral']
new_squished_df = df_squished_emotion[df_squished_emotion['emotions'].isin(desired_emotions)]
new_squished_df

emotions                                               text
1      sadness  Layin n bed with a headache  ughhhh...waitin o...
2      sadness                Funeral ceremony...gloomy friday...
4      neutral  @dannycastillo We want to trade with someone w...
5         fear  Re-pinging @ghostridah14: why didn't you go to...
6      sadness  I should be sleep, but im not! thinking about ...
...        ...                                                ...
39995  neutral                                   @JohnLloydTaylor
39996     love                     Happy Mothers Day  All my love
39997     love  Happy Mother's Day to all the mommies out ther...
39998      joy  @niariley WASSUP BEAUTIFUL!!! FOLLOW ME!!  PEE...
39999     love  @mopedronin bullet train from tokyo    the gf ...

[36465 rows x 2 columns]

In [19]:
# proportions for squished emotions dataset
squished_proportions = get_proportions(new_squished_df['emotions'])
print(squished_proportions)

          count  percentage
neutral    8580   23.529412
fear       8421   23.093377
joy        6943   19.040176
sadness    5142   14.101193
love       3770   10.338681
surprise   2177    5.970108
anger      1432    3.927053


In [20]:
new_squished_df.to_csv(project_folder + '/datasets/kaggle_data_no_dupe_squished_emotions.csv')